# Stock market analysis during the Covid-19 pandemic

#### Summary:
The Covid-19 pandemic had severe impacts on stock markets, but also brought about opportunities. With the help of analyst reports, we can then capitalize on these opportunities. The main question would then be which analyst reports can we trust? This study aims to validate the accuracy of analyst reports, with a focus on Morningstar and Yahoo Finance recommendations. 

## Libraries

In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
import yfinance as yf

## Validate analyst claims 

### Morningstar
Morningstar: 'both companies (Uber and Lyft) will recover from this, although it will take until next year.’

Prediction: Recovery will take until 2021, where recovery is taken to be pre-covid levels (defined as highest price of stock between Jan-Feb 2020 which was just before Covid-19)

In [2]:
#Markets open on 2nd Jan as 1st Jan is public holiday
#Time horizon from Jan 2020 - Apr 2021
data = yf.download("UBER", start="2020-01-02", end="2021-04-30", interval="1d")
data = data.reset_index() #to convert date from index to column 

[*********************100%***********************]  1 of 1 completed


In [3]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-01-02,29.940001,31.000000,29.790001,30.990000,30.990000,20578900
1,2020-01-03,30.620001,31.430000,30.480000,31.370001,31.370001,18822700
2,2020-01-06,31.010000,32.060001,31.000000,31.580000,31.580000,21204700
3,2020-01-07,31.790001,32.840000,31.360001,32.810001,32.810001,30119600
4,2020-01-08,32.730000,34.520000,32.459999,33.930000,33.930000,43944400


In [4]:
#empty dataframe to store results
#recover_df = pd.DataFrame(columns = col)

#function that takes in stock ticker symbol and returns pre covid stock price which is defined as the highest price of the stock
#between Jan-Feb 2020, the earliest date that the stock hits or surpasses pre-covid level and the price. 
def recover(stock):
    data = yf.download(stock, start="2020-01-02", end="2021-04-30", interval="1d")
    data = data.reset_index()
    index = int(data[data['Date'] == '2020-02-28'].index.values) #index for last date of trading in feb 2020
    pre_covid = round(max(data['Close'].iloc[:index]),2)
    col = ['Stock', 'Pre-Covid level', 'Earliest recovery date', 'Price']
    for i in range(index+1,len(data)):
        if data['Close'].iloc[i]>= pre_covid:
            date = data['Date'].iloc[i].date()
            price = round(data['Close'].iloc[i],2)
            break
    try:
        df = pd.Series([stock,'$' + str(pre_covid),date,'$'+str(price)],index = col)
    except:
        price = round(max(data['Close'].iloc[index+1:]),2)
        date = data['Date'].iloc[int(data[data['Close']==max(data['Close'].iloc[index+1:])].index.values)].date()
        df = pd.Series([stock,'$' + str(pre_covid),date,'$'+str(price)],index = col)
    return df
    
col = ['Stock', 'Pre-Covid level', 'Earliest recovery date', 'Price']
recover_df = pd.DataFrame(columns = col)
for i in ['UBER','LYFT']:
    recover_df = recover_df.append(recover(i),ignore_index = True)
recover_df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Stock,Pre-Covid level,Earliest recovery date,Price
0,UBER,$41.27,2020-11-05,$41.96
1,LYFT,$53.94,2021-02-10,$56.21


Prediction that recovery will take until 2021 is partially true. Actual recovery was quicker for Uber that occurred on 5 Nov 2020, but actual recovery was as predicted for Lyft that occurred on 10 Feb 2021 when stock prices recovered to pre-covid levels.

## Fair value estimate
Morningstar provides fair value estimates for stocks which are like reference points that an investor can use to determine if a stock is undervalued or overvalued. 

In [5]:
#empty dataframe to store results
col = ['Stock', 'Current date', 'Current price', 'Fair value estimate', 'Actual price', 'Time to reach fair/max/min value', 
       'Profit','Hit fair value?']
fairvalue_df = pd.DataFrame(columns = col)

#function that takes in the stock ticker symbol, fair value estimate, date of analysis,
#and returns when the fair value estimate is achieved, the profit earned from buying the stock when price < fair value, 
#and if price increases further when price > fair value 
def fairvalue(stock, estimate, date): 
    data = yf.download(stock, start="2020-01-02", end="2021-04-30", interval="1d")
    data = data.reset_index()
    index = data[data['Date'] == date].index.values
    price = np.round(data['Close'].iloc[index].values[0],2)
    if price < estimate: # buy since price < fair value
        if max(data['Close'].iloc[int(index):]) < estimate: # did not hit fair value
            actualprice = round(max(data['Close'].iloc[int(index):]),2)
            maxprice = data[data['Close'] == max(data['Close'].iloc[int(index):])].index.values
            time = [data['Date'].iloc[int(maxprice)].date(), data['Date'].iloc[int(maxprice)] - pd.to_datetime(date) ]
            profit = "{:.0%}".format((actualprice-price)/price)
        else: # hit fair value
            for i in range(int(index),len(data)):
                if data['Close'][i] >= estimate:
                    actualprice = round(data['Close'].iloc[i],2)
                    time = [data['Date'].iloc[i].date(),data['Date'].iloc[i]-pd.to_datetime(date)]
                    profit = "{:.0%}".format((actualprice-price)/price)
                    break
        if actualprice < estimate:
            hit = 'No'
        else:
            hit = 'Yes'
    else: # sell since price > fair value
        for i in range(int(index),len(data)):
            if data['Close'][i] <= estimate: # hit fair value
                actualprice = round(data['Close'][i],2)
                time = [data['Date'][i].date(),data['Date'][i]-pd.to_datetime(date)]
                profit = "{:.0%}".format((price - actualprice)/actualprice)
                break
            else: # did not hit fair value
                actualprice = round(min(data['Close'].iloc[int(index):]),2)
                minprice = data[data['Close'] == min(data['Close'].iloc[int(index):])].index.values
                time = [data['Date'].iloc[int(minprice)].date(), data['Date'].iloc[int(minprice)] - pd.to_datetime(date) ]
                profit = "{:.0%}".format((price - actualprice)/actualprice)
                break
        if actualprice <= estimate:
            hit = 'Yes'
        else:
            hit = 'No'
    df = pd.Series([stock,date,price, estimate, actualprice,time,profit,hit], index = col)
    return df
    
    
    

In [6]:
#analysis from https://www.morningstar.com/articles/972772/5-stocks-we-like
#buy as fair value > current price
stocks = ['FANG', 'FSLR', 'SYY', 'CMCSA']
date = '2020-03-17'
fair_value = [90,59,59,49]
#function that takes in stock ticker, date, fair value and returns output of fairvalue function defined above
def morningstar(stocks, date, fair_value, df):
    stock_dict = {'stocks': stocks, 'date': date, 'fair_value': fair_value}
    for i in range(len(stock_dict['stocks'])):
        stock = stock_dict['stocks'][i]
        estimate = stock_dict['fair_value'][i]
        series = fairvalue(stock,estimate,date)
        df = df.append(series,ignore_index = True)
    return df
fairvalue1 = morningstar(stocks,date,fair_value,fairvalue_df)

#analysis from https://www.morningstar.com/articles/973345/5-more-stocks-we-like
#buy as fair value > current price
stocks = ['CVE','BUD','CTVA','AXP','ECL']
date = '2020-03-19'
fair_value = [7,96,40,125,191]
fairvalue2 = morningstar(stocks, date, fair_value, fairvalue1)

#analysis from https://www.morningstar.com/articles/976067/6-more-stocks-we-like (stock name 'CTL' changed to 'LUMN')
#buy as fair value > current price
stocks = ['CLR', 'LUMN', 'VVV', 'INGR', 'TWNK', 'GOOG']
date = '2020-04-02'
fair_value = [19,18,19.8,125,15.8,1400]
stock_dict = {'stocks':['CLR', 'LUMN', 'VVV', 'INGR', 'TWNK', 'GOOG'], 'date':'2020-04-02', 
              'fair_value':[19,18,19.8,125,15.8,1400]}
fairvalue3 = morningstar(stocks, date, fair_value, fairvalue2)

#analysis from https://www.morningstar.com/articles/999855/3-expensive-diy-stocks-to-watch
# sell as fair value < current price
stocks = ['HD', 'LOW', 'SHW']
date = '2020-08-28'
fair_value = [200, 133, 360]
stock_dict = {'stocks':['HD', 'LOW', 'SHW'], 'date':'2020-08-28', 'fair_value':[200, 133, 360]}
fairvalue_final = morningstar(stocks, date, fair_value, fairvalue3)

#add column to check if difference between actual price and fair value estimate exceeds 15% of the fair value
fairvalue_final['>15percent difference'] = ' '
for i in range(len(fairvalue_final)):
    if fairvalue_final['Hit fair value?'].iloc[i] == 'No':
        diff = fairvalue_final['Fair value estimate'].iloc[i] - fairvalue_final['Actual price'].iloc[i]
        if abs(diff) > 0.15*fairvalue_final['Fair value estimate'].iloc[i]:
            fairvalue_final['>15percent difference'].iloc[i] = 'Yes'
        else:
            fairvalue_final['>15percent difference'].iloc[i] = 'No'
        
fairvalue_final.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

C:\Users\User\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Stock,Current date,Current price,Fair value estimate,Actual price,Time to reach fair/max/min value,Profit,Hit fair value?,>15percent difference
0,FANG,2020-03-17,20.23,90,85.27,"[2021-03-05, 353 days 00:00:00]",322%,No,No
1,FSLR,2020-03-17,32.26,59,61.78,"[2020-07-14, 119 days 00:00:00]",92%,Yes,
2,SYY,2020-03-17,35.06,59,59.10,"[2020-06-03, 78 days 00:00:00]",69%,Yes,
3,CMCSA,2020-03-17,38.22,49,49.00,"[2020-11-13, 241 days 00:00:00]",28%,Yes,
4,CVE,2020-03-19,1.64,7,7.19,"[2021-02-22, 340 days 00:00:00]",338%,Yes,


In [7]:
fairvalue_final

,Stock,Current date,Current price,Fair value estimate,Actual price,Time to reach fair/max/min value,Profit,Hit fair value?,>15percent difference
0,FANG,2020-03-17,20.23,90,85.27,"[2021-03-05, 353 days 00:00:00]",322%,No,No
1,FSLR,2020-03-17,32.26,59,61.78,"[2020-07-14, 119 days 00:00:00]",92%,Yes,
2,SYY,2020-03-17,35.06,59,59.10,"[2020-06-03, 78 days 00:00:00]",69%,Yes,
3,CMCSA,2020-03-17,38.22,49,49.00,"[2020-11-13, 241 days 00:00:00]",28%,Yes,
4,CVE,2020-03-19,1.64,7,7.19,"[2021-02-22, 340 days 00:00:00]",338%,Yes,
5,BUD,2020-03-19,36.36,96,72.14,"[2021-04-29, 406 days 00:00:00]",98%,No,Yes
6,CTVA,2020-03-19,23.26,40,41.07,"[2021-01-06, 293 days 00:00:00]",77%,Yes,
7,AXP,2020-03-19,77.07,125,125.04,"[2020-12-04, 260 days 00:00:00]",62%,Yes,
8,ECL,2020-03-19,159.58,191,197.58,"[2020-04-28, 40 days 00:00:00]",24%,Yes,
9,CLR,2020-04-02,8.34,19,21.58,"[2020-06-08, 67 days 00:00:00]",159%,Yes,


In [8]:
mean_profit = []
for i in fairvalue_final['Profit']:
    mean_profit.append(int(i.strip('%')))
count = 0
for i in mean_profit:
    count += i
print('average profit: ' + str(count/len(mean_profit)))
avg_date = fairvalue_final['Time to reach fair/max/min value'][0][1]
for i in fairvalue_final['Time to reach fair/max/min value'].iloc[1:]:
    avg_date += i[1]
print('average time: ' + str(avg_date/len(fairvalue_final)))

#compare to S&P 500 returns for same duration of 216 days starting on same date as first analysis
end_date = str(datetime.date(2020,3,17) + timedelta(days=216))
data = yf.download("SPY", start="2020-03-17", end=end_date, interval="1d")
data = data.reset_index()
print('S&P 500 return: ' + str("{:.0%}".format((data['Close'].iloc[-1]-data['Close'].iloc[0])/data['Close'].iloc[0])))

average profit: 86.11111111111111
average time: 216 days 06:40:00
[*********************100%***********************]  1 of 1 completed
S&P 500 return: 45%


## yfinance recommendations 
The package yfinance allows users to download historical market data from yahoo finance. It also provides recommendations which will be validated. 

In [9]:
#function for data cleaning. Takes in ticker symbol as input and outputs data from Jan 2020 - Apr 2021
def clean(stock):
    stock_df = yf.Ticker(stock)
    stock_df = stock_df.recommendations.reset_index()
    stock_df['Date'] = pd.to_datetime(stock_df['Date']).dt.date #get rid of the time and keep the date portion 
    for i in range(len(stock_df)):
        if stock_df['Date'][i].year==2020:
            index_1 = i 
            break
    for k in range(len(stock_df)):
        if stock_df['Date'][k].year==2021 and stock_df['Date'][k].month==5:
            index_2 = k
            break
    try:
        stock_df = stock_df.iloc[index_1:index_2]
    except: # error due to no recommendations in May 2021, so take last recommendation
        stock_df = stock_df.iloc[index_1:]
    return stock_df


In [10]:
#Example from Uber that shows the recommendations by investment research firms 
uber_df = clean('UBER')
uber_df.head()

,Date,Firm,To Grade,From Grade,Action
47,2020-01-10,Bernstein,Outperform,,init
48,2020-01-23,Guggenheim,Buy,,main
49,2020-01-28,UBS,Buy,,init
50,2020-01-31,JP Morgan,Overweight,,init
51,2020-02-03,Wedbush,Outperform,,main


### IT sector

In [11]:
#Top 30 companies in terms of market cap from the S&P 500 Information Technology. It comprises the companies (75 constituents) 
#included in the S&P 500 that are classified as members of the GICS information technology sector.
#Data taken as of May 2021 from https://www.tradingview.com/symbols/SP-S5INFT/components/

itlist = ['AAPL','MSFT','V','NVDA','MA','PYPL','ADBE','INTC', 'ORCL','CSCO','CRM','ACN','AVGO','TXN','QCOM','IBM','AMAT','INTU',
         'AMD','NOW','FIS','MU','LRCX','ADP','FISV','ADSK','ADI','GPN','NXPI','KLAC']

In [12]:
#function to get unique recommendations  
def unique(sector_list):
    sector_dict = {}
    for i in sector_list:
        sector_dict[i] = clean(i)
    unique_list_1 = []
    for i in sector_dict:
        unique_list_1.append(sector_dict[i]['To Grade'].unique())
    unique_list_2 = []
    for i in range(len(unique_list_1)):
        for k in unique_list_1[i]:
            unique_list_2.append(k)
    unique_list_3 = pd.Series(unique_list_2).unique()
    print('unique recommendations: ' + str(unique_list_3))
    return(sector_dict)


In [13]:
sector_dict = unique(itlist)

unique recommendations: ['Outperform' 'Buy' 'Underweight' 'Overweight' 'Neutral' 'Hold'
 'Equal-Weight' 'Underperform' 'Market Perform' 'Sell' 'Sector Perform'
 'Strong Buy' 'Positive' 'Sector Weight' 'Perform' 'Reduce'
 'Market Outperform' 'In-Line' 'Peer Perform' 'Top Pick']


In [14]:
#sort into buy, neutral, sell categories 
buy_list = ['Outperform','Buy','Overweight','Strong Buy','Positive','Market Outperform','Top Pick']
neutral_list = ['Neutral','Hold','Equal-Weight','Market Perform','Sector Perform', 'Sector Weight','Perform','In-Line',
               'Peer Perform']
sell_list = ['Underweight','Underperform','Sell','Reduce']

In [15]:
#function that takes in a list of dates and returns the average date 
def avg(dates):
    any_reference_date = datetime.datetime(2020, 2, 28)
    return any_reference_date + (sum([date - any_reference_date for date in dates],datetime.timedelta()) / len(dates))

#function that returns whether firm recovers to pre-covid price and time taken to recover
def recommend(sector_dict,buy_list,neutral_list,sell_list):
    # Total number of recommendations for each category 
    buy_counter = 0
    neutral_counter = 0
    sell_counter = 0
    for i in sector_dict:
        for k in range(len(sector_dict[i]['To Grade'])):
            if sector_dict[i]['To Grade'].iloc[k] in buy_list:
                buy_counter += 1
            elif sector_dict[i]['To Grade'].iloc[k] in neutral_list:
                neutral_counter += 1
            elif sector_dict[i]['To Grade'].iloc[k] in sell_list:
                sell_counter += 1 
    print('buy: ' + str(buy_counter), 'neutral: ' + str(neutral_counter), 'sell: ' + str(sell_counter))
    
    #which firm had the most neutral/sell recommendations?
    col = ['Stock', 'Number of buy recommendations', 'Number of neutral recommendations', 'Number of sell recommendations']
    firm_df = pd.DataFrame(columns = col)
    for i in sector_dict:
        stock = i
        buy_count = 0
        neutral_count = 0
        sell_count = 0
        for k in range(len(sector_dict[i]['To Grade'])):
            if sector_dict[i]['To Grade'].iloc[k] in buy_list:
                buy_count+=1
            elif sector_dict[i]['To Grade'].iloc[k] in neutral_list:
                neutral_count+=1
            elif sector_dict[i]['To Grade'].iloc[k] in sell_list:
                sell_count+=1
        df = pd.Series([stock,buy_count,neutral_count,sell_count], index = col)
        firm_df = firm_df.append(df, ignore_index = True)
    firm_df['buy:neutral+sell'] = ' '
    for i in range(len(firm_df)):
        if (firm_df['Number of neutral recommendations'].iloc[i] + firm_df['Number of sell recommendations'].iloc[i]) == 0:
             firm_df['buy:neutral+sell'].iloc[i] = firm_df['Number of buy recommendations'].iloc[i]/1
        else:
            firm_df['buy:neutral+sell'].iloc[i] = firm_df['Number of buy recommendations'].iloc[i]/(
                firm_df['Number of neutral recommendations'].iloc[i] + firm_df['Number of sell recommendations'].iloc[i])
    
    #split firms with more neutral and sell recommendations than buy recommendations 
    firm_negative = [] # more neutral and sell recommendations 
    firm_positive = [] # more buy recommendations
    for i in range(len(firm_df['Stock'])):
        if firm_df['buy:neutral+sell'].iloc[i]<1:
            firm_negative.append(firm_df['Stock'].iloc[i])
        else:
            firm_positive.append(firm_df['Stock'].iloc[i])
    col = ['Stock', 'Pre-Covid level', 'Earliest recovery date', 'Price']
    recover_df = pd.DataFrame(columns = col)
    for i in firm_df['Stock']:
        recover_df = recover_df.append(recover(i),ignore_index = True)
    
    # number of firms which did not recover to pre-covid level out of 30 total companies
    count = 0
    stock_list = []
    for i in range(len(recover_df)):
        if recover_df['Price'].iloc[i] < recover_df['Pre-Covid level'].iloc[i]:
            count += 1
            stock_list.append(recover_df['Stock'].iloc[i])
    print('number of firms that did not recover to pre-covid level: '+ str(count))
    
    #number of firms which did not recover to pre-covid level and has more sell/neutral than buy recommendations
    count = 0
    for i in stock_list:
        if i in firm_negative:
            count += 1
    print('number of firms with more sell/neutral than buy recommendations: '+ str(count))
    
    negative_df = pd.DataFrame(columns = col)
    positive_df = pd.DataFrame(columns = col)
    for i in firm_negative:
        negative_df = negative_df.append(recover_df.iloc[int(recover_df[recover_df['Stock']==i].index.values)], ignore_index = True)
    for i in firm_positive:
        positive_df = positive_df.append(recover_df.iloc[int(recover_df[recover_df['Stock']==i].index.values)], ignore_index = True)
    
    #function that takes in a list of dates and returns the average date 
    print('average date to reach pre-covid/max level for positive firms: ' + 
      str(avg(pd.to_datetime(positive_df['Earliest recovery date']))))
    print('average date to reach pre-covid/max level for negative firms: ' + 
      str(avg(pd.to_datetime(negative_df['Earliest recovery date']))))

In [16]:
recommend(sector_dict,buy_list,neutral_list,sell_list)

buy: 1488 neutral: 430 sell: 63
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*

### Healthcare sector

In [17]:
#Top 30 companies in terms of market cap from the S&P 500 Healthcare.
#Data taken as of May 2021 from https://www.tradingview.com/symbols/SP-S5HLTH/components/
healthlist = ['JNJ','UNH','PFE','ABBV','ABT','LLY','MRK','TMO','DHR','MDT','BMY','AMGN','CVS','ISRG','ANTM','SYK','CI','GILD',
              'ZTS','HCA','BDX','ILMN','BSX','EW','HUM','REGN','VRTX','IDXX','ALGN','IQV']

In [18]:
health_dict = unique(healthlist)

unique recommendations: ['Overweight' 'Outperform' 'Buy' 'Hold' 'Neutral' 'Strong Buy'
 'Market Perform' 'Equal-Weight' 'Sector Perform' 'Peer Perform'
 'Sector Weight' 'Sell' 'Underperform' 'Underweight' 'Perform' 'In-Line'
 'Market Outperform']


In [19]:
#sort into buy,neutral,sell categories
buy_list = ['Overweight','Outperform','Buy','Strong Buy','Market Outperform']
neutral_list = ['Hold','Neutral','Market Perform','Equal-Weight','Sector Perform','Peer Perform','Sector Weight',
                'Perform', 'In-Line']
sell_list = ['Sell','Underperform','Underweight']


In [20]:
recommend(health_dict,buy_list,neutral_list,sell_list)

buy: 766 neutral: 275 sell: 22
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[**

### Financials sector

In [21]:
#Top 30 companies in terms of market cap from the S&P 500 Financials.
#Data taken as of May 2021 from https://www.tradingview.com/symbols/SP-SPF/components/
finlist = ['BRK-B','JPM','BAC','WFC','MS','C','SCHW','BLK','GS','AXP','SPGI','USB','PNC','CME','CB','COF','MMC','ICE','MCO',
           'PGR','MET','AON','AIG','BK','TROW','PRU','TRV','AFL','MSCI','DFS']

In [22]:
fin_dict = unique(finlist)

unique recommendations: ['Buy' 'Hold' 'Equal-Weight' 'Peer Perform' 'Neutral' 'Outperform'
 'Overweight' 'Market Perform' 'Underweight' 'Underperform' 'Sell'
 'Sector Perform' 'Conviction Buy' 'Market Outperform' 'Perform'
 'Strong Buy' 'In-Line']


In [23]:
#sort into buy,neutral,sell categories
buy_list = ['Buy','Overweight','Outperform','Conviction Buy','Market Outperform','Strong Buy']
neutral_list = ['Hold','Equal-Weight','Peer Perform','Neutral','Market Perform', 'Sector Perform','Perform','In-Line']
sell_list = ['Sell','Underperform','Underweight']


In [24]:
recommend(fin_dict,buy_list,neutral_list,sell_list)

buy: 540 neutral: 464 sell: 78
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[**